# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298582522352                   -0.85    4.5
  2   -8.300209237857       -2.79       -1.25    1.0
  3   -8.300446654437       -3.62       -1.87    4.4
  4   -8.300462202952       -4.81       -2.73    1.4
  5   -8.300464268939       -5.68       -3.07    4.5
  6   -8.300464448515       -6.75       -3.21    4.0
  7   -8.300464549117       -7.00       -3.37    5.8
  8   -8.300464592490       -7.36       -3.48    5.2
  9   -8.300464631578       -7.41       -3.69    1.1
 10   -8.300464631447   +   -9.88       -3.71    1.2
 11   -8.300464640574       -8.04       -3.93    2.1
 12   -8.300464643932       -8.47       -4.50    1.4


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64212242655                   -0.70    5.4
  2   -16.64609581056       -2.40       -1.14    1.2
  3   -16.67922200763       -1.48       -1.86    3.5
  4   -16.67928186493       -4.22       -2.68    2.9
  5   -16.67928606684       -5.38       -3.00    6.0
  6   -16.67928621304       -6.84       -3.30    1.6
  7   -16.67928622263       -8.02       -3.77    1.0
  8   -16.67928622414       -8.82       -4.50    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32023150354                   -0.56    6.4
  2   -33.32794612948       -2.11       -1.00    1.0
  3   -33.33677073886       -2.05       -1.68    4.9
  4   -33.33691671530       -3.84       -2.58    2.5
  5   -33.33694096928       -4.62       -3.13    8.6
  6   -33.33694384796       -5.54       -3.82    4.4
  7   -33.33694392545       -7.11       -4.51    8.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298562337542                   -0.85    4.0
  2   -8.300287137231       -2.76       -1.59    1.5
  3   -8.300463114632       -3.75       -2.71    5.9
  4   -8.300462535640   +   -6.24       -3.13    6.2
  5   -8.300464036719       -5.82       -3.36    6.6
  6   -8.300464574164       -6.27       -3.85    2.1
  7   -8.300464643583       -7.16       -5.08    3.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32566552525                   -0.56    6.1
  2   -33.32362588062   +   -2.69       -1.27    1.0
  3   -20.80872480586   +    1.10       -0.52    8.1
  4   -33.29854186467        1.10       -1.74    7.8
  5   -33.15335572811   +   -0.84       -1.36    4.0
  6   -32.91137353361   +   -0.62       -1.25    5.2
  7   -33.32032333167       -0.39       -1.86    5.0
  8   -33.33624949907       -1.80       -2.45    2.4
  9   -33.33637557747       -3.90       -2.54    3.4
 10   -33.33652278955       -3.83       -2.58    2.4
 11   -33.33685237337       -3.48       -2.91    1.9
 12   -33.33693193396       -4.10       -3.29    3.2
 13   -33.33693785123       -5.23       -3.45    3.0
 14   -33.33694275412       -5.31       -3.76    2.2
 15   -33.33694128275   +   -5.83       -3.84    4.9
 16   -33.33694248449       -5.92       -3.97    3.1
 17   -33.33694384676       -5.87       -4.51 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.